In [1]:
import requests
import cv2

In [2]:
from bs4 import BeautifulSoup
import re
import os
import time
import pandas as pd
import numpy as np

In [10]:
url = 'https://this-person-does-not-exist.com'
path = './data/generated'

In [3]:
retrieve_n = 5
retrieved = 0

for image in range(retrieve_n):
    
    res = requests.get(url)
    good_status = res.status_code == 200
    wait_cycle = 0
    failed_connection = False
    
    while not good_status:
        wait_cycle += 1
        print('Site did not respond. Retrying...')
        if wait_cycle > 5:
            print(f'Could not reach site. Terminating request. \
            Loop collected {retrieved} out of {retrieve_n} image(s) prior to lost connection.')
            failed_connection = True
            break
        time.sleep(3)
        res = requests.get(url)
        good_status = res.status_code == 200

    if failed_connection:
        break

    soup = BeautifulSoup(res.text, 'lxml')

    if not os.path.exists(path):
        os.mkdir(path)
    
    image_json = soup.find('img', {'id': 'avatar'})
    image_link = url + image_json['src']
    image_name = re.findall("avatar\S+.jpg", image_link)[0]
    image_path = path + '/' + image_name
    image_res = requests.get(image_link)

    if image_res.status_code != 200:
        continue

    file = open(image_path, 'wb')
    file.write(image_res.content)
    file.close()
    
    retrieved += 1

print(f'Retrieved {retrieved} of targeted {retrieve_n} images.')

Retrieved 5 of targeted 5 images.


In [3]:
for i in range(5):
    
    wait_cycle = 0
    failed_connection = False
    
    while 100 != 200:
        wait_cycle += 1
        print('Site did not respond. Retrying...')
        if wait_cycle > 5:
            print(f'Could not reach site. Terminating request. Loop collected {i} image(s) prior to lost connection.')
            failed_connection = True
            break
        time.sleep(1)

    if failed_connection:
        break
    print(i)

Site did not respond. Retrying...
Site did not respond. Retrying...
Site did not respond. Retrying...
Site did not respond. Retrying...
Site did not respond. Retrying...
Site did not respond. Retrying...
Could not reach site. Terminating request. Loop collected 0 image(s) prior to lost connection.


In [4]:
# Image resizing from: https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/

img1 = cv2.imread('./data/generated/avatar-008390911eb28a25b6dcbbea9cb607ac.jpg', cv2.IMREAD_UNCHANGED)
img1[:2]

array([[[ 98, 101, 109],
        [ 77,  80,  88],
        [ 72,  75,  83],
        ...,
        [255, 245, 238],
        [255, 241, 235],
        [230, 214, 207]],

       [[ 81,  81,  87],
        [ 53,  51,  57],
        [ 63,  59,  65],
        ...,
        [255, 247, 240],
        [253, 239, 233],
        [249, 233, 227]]], dtype=uint8)

In [5]:
width = 252
height = 252
dim = (width, height)
resized = cv2.resize(img1, dim, interpolation = cv2.INTER_AREA)

In [6]:
cv2.imshow('Image 1', resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

image_json = soup.find('img', {'id': 'avatar'})
image_link = url + image_json['src']
image_name = re.findall("avatar\S+.jpg", image_link)[0]
image_path = path + '/' + image_name
image_res = requests.get(image_link, stream=True).raw

# raw image handling from: https://stackoverflow.com/questions/57539233/how-to-open-an-image-from-an-url-with-opencv-using-requests-from-python
image = np.asarray(bytearray(image_res.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

width = 252
height = 252
dim = (width, height)
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

cv2.imshow('Image 1', resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
pd.read_csv('https://www.cs.columbia.edu/CAVE/databases/pubfig/download/eval_urls.txt', sep='\t')

# PubFig Dataset v1.2 - eval_urls.txt - http://www.cs.columbia.edu/CAVE/databases/pubfig/
#             person imagenum                                           url              rect                                                                         md5sum                                       
Aaron Eckhart 1      http://farm1.static.flickr.com/119/288329997_19... 248,92,338,182   a980a9e21c90ff62e57345fad53a56c8                                                NaN                                       
              2      http://farm1.static.flickr.com/35/99344798_f2ad... 267,138,419,290  8b2bc3a7a3b4a9d5826cd31ac9254924                                                NaN                                       
              4      http://2.bp.blogspot.com/_DxSfGxvclek/SH-K403dD... 32,39,94,101     994b9bfd1464936488458d2679e05520                                                NaN                                       
              5      http://2.bp.blogspot.com/_biK-MLwOHEc/RtXJ3nA4y... 75,66,183,174    d50214036344a000cdf1e68832acf33f                                                NaN                                       
...                                                                                                                                                                      ...                                       
William Macy  96     http://farm1.static.flickr.com/62/207526936_d9d... 504,174,694,364  2df2257b04cd4f3c23ce95d67e867e2c                                                NaN                                       
              97     http://farm1.static.flickr.com/98/243909925_1ae... 275,182,429,336  af91f80483ef3aa34ca2f80ab91c09b3                                                NaN                                       
              98     http://farm1.static.flickr.com/91/245712735_4d8... 908,564,1200,857 bd1e84385beed6c5dca4f90d044ddda3                                                NaN                                       
              99     http://farm1.static.flickr.com/33/39937487_72ab... 373,259,479,365  d724cfd1decebac4d6ee79e799807f88                                                NaN                                       
              100    http://farm1.static.flickr.com/31/39937508_3fec... 180,298,442,560  d1910cdf30aa90c3436d69c6e18e35f4                                                NaN                                       

[42462 rows x 1 columns]

In [30]:
real_link = 'https://www.cs.columbia.edu/CAVE/databases/pubfig/download/eval_urls.txt'
real_resp = requests.get(real_link)
output = open('./data/real/real_images.txt', 'wb')
output.write(real_resp.content)
output.close()

In [31]:
# Approach to delete lines found here: https://pynative.com/python-delete-lines-from-file/

with open('./data/real/real_images.txt', 'r') as input_data:
    lines = input_data.readlines()

In [32]:
lines[:2]

['# PubFig Dataset v1.2 - eval_urls.txt - http://www.cs.columbia.edu/CAVE/databases/pubfig/\n',
 '#\tperson\timagenum\turl\trect\tmd5sum\n']

In [33]:
with open('./data/real/real_images.txt', 'w') as input_data:
    for number, line in enumerate(lines):
        if number == 1:
            input_data.write(line.strip('#\t'))
        else:
            if number not in [0]:
                input_data.write(line)

In [34]:
with open('./data/real/real_images.txt', 'r') as input_data:
    lines = input_data.readlines()
lines[:2]

['person\timagenum\turl\trect\tmd5sum\n',
 'Aaron Eckhart\t1\thttp://farm1.static.flickr.com/119/288329997_19ebf1d7b3_o.jpg\t248,92,338,182\ta980a9e21c90ff62e57345fad53a56c8\n']

In [15]:
df_real = pd.read_csv('./data/real/real_images.txt', sep='\t')
df_real.head()

,person,imagenum,url,rect,md5sum
0,Aaron Eckhart,1,http://farm1.static.flickr.com/119/288329997_1...,"248,92,338,182",a980a9e21c90ff62e57345fad53a56c8
1,Aaron Eckhart,2,http://farm1.static.flickr.com/35/99344798_f2a...,"267,138,419,290",8b2bc3a7a3b4a9d5826cd31ac9254924
2,Aaron Eckhart,4,http://2.bp.blogspot.com/_DxSfGxvclek/SH-K403d...,"32,39,94,101",994b9bfd1464936488458d2679e05520
3,Aaron Eckhart,5,http://2.bp.blogspot.com/_biK-MLwOHEc/RtXJ3nA4...,"75,66,183,174",d50214036344a000cdf1e68832acf33f
4,Aaron Eckhart,6,http://3.bp.blogspot.com/_bto58WjLomw/Rk1yrB-r...,"65,66,133,134",626836ef2e4c2afdcaa6ad8bc1842778


In [34]:
real_link = df_real['url'].sample(random_state=17).values[0]
real_link

'http://fanphotosource.com/harrison-ford/photos/harrison-ford-and_elizabeth-taylor/harrison-ford-elizabeth-taylor-0000101.jpg'

In [36]:
real_link = 'http://www.firstshowing.net/img/harvey-dent-believe-350w.jpg'

In [44]:
df_real.loc[0, 'rect']

'248,92,338,182'

In [49]:
rect = '237,102,333,198'
rect_vals = [int(x) for x in rect.split(',')]
rect_vals

[237, 102, 333, 198]

In [69]:
face_center = [(rect_vals[0]-rect_vals[1])//2 + rect_vals[1],(rect_vals[2]-rect_vals[3])//2 + rect_vals[3]]
face_center

[169, 265]

In [60]:
rect_height = rect_vals[0] - rect_vals[1]
rect_width = rect_vals[2] - rect_vals[3]
rect_height, rect_width

(135, 135)

In [77]:
padded_height = rect_height * 15//10
padded_height

202

In [78]:
padded_width = rect_width * 15//10
padded_width

202

In [67]:
px_square = max(padded_height, padded_width)
px_square

148

In [89]:
image_link = real_link
image_res = requests.get(image_link, stream=True).raw

# raw image handling from: https://stackoverflow.com/questions/57539233/how-to-open-an-image-from-an-url-with-opencv-using-requests-from-python
image = np.asarray(bytearray(image_res.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

# crop image
rect = '237,102,333,198' # this will be brought in from the dataframe
rect_vals = [int(x) for x in rect.split(',')]
face_center = [(rect_vals[0]-rect_vals[1])//2 + rect_vals[1],(rect_vals[2]-rect_vals[3])//2 + rect_vals[3]]
rect_height = rect_vals[0] - rect_vals[1]
rect_width = rect_vals[2] - rect_vals[3]
padded_height = rect_height * 15//10
padded_width = rect_width * 15//10
px_square = max(padded_height, padded_width) # produce a square that is sure to capture target face
offset = px_square//2

image = image[face_center[0]-offset:face_center[0]+offset, face_center[1]-offset:rect_vals[2]+offset]

# cv2.imshow('Image 1', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

width = 252
height = 252
dim = (width, height)
# resized = cv2.resize(image, dim, interpolation = cv2.INTER_NEAREST) # Pretty bad
# resized = cv2.resize(image, dim, interpolation = cv2.INTER_LINEAR) # Decent, but a little blurry
# resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) # Decent, but pixelated
# resized = cv2.resize(image, dim, interpolation = cv2.INTER_CUBIC) # Impressive!
resized = cv2.resize(image, dim, interpolation = cv2.INTER_LANCZOS4) # Very impressive!

cv2.imshow('Image 1', resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
df_real['url'].values[:5]

array(['http://farm1.static.flickr.com/119/288329997_19ebf1d7b3_o.jpg',
       'http://farm1.static.flickr.com/35/99344798_f2ad604eda_o.jpg',
       'http://2.bp.blogspot.com/_DxSfGxvclek/SH-K403dDuI/AAAAAAAAAC8/6TGJ67y8kZk/s320/Aaron%2BEckhart.jpg',
       'http://2.bp.blogspot.com/_biK-MLwOHEc/RtXJ3nA4y5I/AAAAAAAAALM/fMGxP5sRK10/s320/aaroneckhart.jpg',
       'http://3.bp.blogspot.com/_bto58WjLomw/Rk1yrB-rEXI/AAAAAAAABGo/dy33tJJsjSE/s400/aaron_eckhart3.jpg'],
      dtype=object)

In [32]:
# broken_link_indices = []

# for index, link in df_real['url'].values:
#     try:
#         requests.get(image_link, stream=True).raw
#     except:
#         broken_link_indices.append(index)



0 is less than 5
1 is less than 5
2 is less than 5
3 is less than 5
4 is less than 5
5 is not less than 5!
6 is not less than 5!
7 is not less than 5!
8 is not less than 5!
9 is not less than 5!
